# Plugins Quickstart

[What is a Plugin?](https://learn.microsoft.com/en-us/semantic-kernel/concepts/plugins/?pivots=programming-language-python#importing-different-types-of-plugins)

In [76]:
# ! python.exe -m pip install --upgrade pip

In [77]:
! pip install -q -r requirements.txt

In [1]:
import os
from dotenv import load_dotenv

from typing import TypedDict, Annotated, List, Optional

import asyncio

from semantic_kernel import Kernel
from semantic_kernel.functions import kernel_function
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, AzureChatPromptExecutionSettings
from semantic_kernel.connectors.ai import FunctionChoiceBehavior
from semantic_kernel.contents import ChatHistory, ChatMessageContent
from semantic_kernel.contents.utils.author_role import AuthorRole
#from semantic_kernel.functions import KernelArguments

In [2]:
#load variables
load_dotenv()

# Variables - Azure Services
azopenai_ep=os.environ["AZURE_OPENAI_ACCOUNT"]
azopenai_key=os.environ["AZURE_OPENAI_KEY"]
azopenai_model=os.environ["AZURE_OPENAI_MODEL"]

In [3]:
class LightModel(TypedDict):
   id: int
   name: str
   is_on: bool | None
   brightness: int | None
   hex: str | None

class LightsPlugin:
   lights: list[LightModel] = [
      {"id": 1, "name": "Table Lamp", "is_on": False, "brightness": 100, "hex": "FF0000"},
      {"id": 2, "name": "Porch light", "is_on": False, "brightness": 50, "hex": "00FF00"},
      {"id": 3, "name": "Chandelier", "is_on": True, "brightness": 75, "hex": "0000FF"},
   ]

   @kernel_function
   async def get_lights(self) -> List[LightModel]:
      """Gets a list of lights and their current state."""
      return self.lights

   @kernel_function
   async def get_state(
      self,
      id: Annotated[int, "The ID of the light"]
   ) -> Optional[LightModel]:
      """Gets the state of a particular light."""
      for light in self.lights:
         if light["id"] == id:
               return light
      return None

   @kernel_function
   async def change_state(
      self,
      id: Annotated[int, "The ID of the light"],
      new_state: LightModel
   ) -> Optional[LightModel]:
      """Changes the state of the light."""
      for light in self.lights:
         if light["id"] == id:
               light["is_on"] = new_state.get("is_on", light["is_on"])
               light["brightness"] = new_state.get("brightness", light["brightness"])
               light["hex"] = new_state.get("hex", light["hex"])
               return light
      return None

In [4]:
# Initialize the kernel
kernel = Kernel()

# Add Azure OpenAI chat completion
chat_completion = AzureChatCompletion(
   deployment_name=azopenai_model,
   api_key=azopenai_key,
   endpoint=azopenai_ep
)
kernel.add_service(chat_completion)

# Add a plugin (the LightsPlugin class is defined below)
kernel.add_plugin(
   LightsPlugin(),
   plugin_name="Lights",
)

# Enable planning
execution_settings = AzureChatPromptExecutionSettings()
execution_settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

# Create a history of the conversation
history = ChatHistory()
history.add_message(
   ChatMessageContent(
      role=AuthorRole.USER,
      content="Tell me the current state of the lamp, then turn on the lamp"
   )
)

# Get the response from the AI
result = await chat_completion.get_chat_message_content(
   chat_history=history,
   settings=execution_settings,
   kernel=kernel,
)

# Print the results
print("Assistant: " + str(result))

# Add the message from the agent to the chat history
history.add_message(result)

Assistant > The "Table Lamp" is already turned on with a brightness of 100. Let me know if you want any adjustments!


In [7]:
# Create a history of the conversation
history = ChatHistory()
history.add_message(
   ChatMessageContent(
      role=AuthorRole.USER,
      content="Tell me the current state of the lamp, then turn off the lamp"
   )
)

# Get the response from the AI
result = await chat_completion.get_chat_message_content(
   chat_history=history,
   settings=execution_settings,
   kernel=kernel,
)

# Print the results
print("Assistant: " + str(result))

# Add the message from the agent to the chat history
history.add_message(result)

Assistant: The current state of the "Table Lamp" is as follows:
- **State**: OFF
- **Brightness**: 100
- **Color**: #FF0000 (red)

Since the lamp is already off, there's no need to turn it off again. Let me know if you'd like further assistance!
